In [1]:
import numpy as np
import pandas as pd
import mlflow
import os
import matplotlib.pyplot as plt
import matplotlib
import math
matplotlib.rcParams['figure.figsize'] = (12.0, 10.0)
from sklearn.metrics import ConfusionMatrixDisplay

from bokeh.plotting import show, output_file, output_notebook
import nntools.report as R


In [2]:
savepoint = 'http://localhost:5010'
exp_name = 'FundusSegmentation'
mlflow.set_tracking_uri(savepoint)

In [3]:
df = mlflow.search_runs('9')

In [4]:
from bokeh.palettes import Category20c
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
output_file('vbar.html')


runs = list(df['tags.mlflow.runName'])
# runs = list(df['run_id'])

mIoUs = list(df['metrics.Test_mIoU'])

source = ColumnDataSource(data=dict(runs=runs, mIoUs=mIoUs, color=Category20c[12]))
p = figure(x_range=runs, plot_height=750, plot_width=750, title="mIoU counts",
           toolbar_location=None, tools="",
          y_range=(0.5, 0.75))

p.vbar(x='runs', top='mIoUs', width=0.9, color='color', source=source)
p.xaxis.major_label_orientation = math.pi/4
output_notebook()
show(p)


Loading BokehJS ...

In [39]:
runs_confMat = []
preds_folder = '/usagers/clpla/Projects/runs/FundusSegmentationProject/'
for index, run in df.iterrows():    
    if not run['status'] == 'FINISHED':
        continue
    run_id = run['run_id']
    
    run_name = run['tags.mlflow.runName']
    path = os.path.join(preds_folder, run_name, 'predictions', run_id, 'confMat.npy')
    runs_confMat.append({'run_name':run_name, 'confmat':np.load(path), 'run_id':run_id})
    path = os.path.join(preds_folder, run_name, 'predictions', run_id, 'confMat.npy')

In [40]:
labels = pd.read_csv('/home/travail/clement/messidor/clean/labels_id.csv')
classes = ['Cotton_Wool_Spot', 'Exudates', 'Hemorrhages', 'Microaneurysms']

In [41]:
source = {}
c = 1
for r in runs_confMat:
    if 'Unet' in r['run_name'] and 'Unet-resnet' not in r['run_name']: 
        source[r['run_name']]= r['confmat'][c]

p = R.display_confMat(source, ['Negative', 'Positive (%s)'%classes[c]], plot_size=750)

output_notebook()
show(p)


Loading BokehJS ...

In [64]:
from bokeh.models import ColumnDataSource, FactorRange
from bokeh.palettes import Spectral6, Spectral5, Spectral11, Category20c
from bokeh.transform import factor_cmap
source = dict(classes=classes)
runs = []
for r in runs_confMat:
    r_name = r['run_name']
    if 'Unet' in r['run_name'] and 'Unet-resnet' not in r['run_name']:
        pass
    else:
        continue

    confMats = r['confmat']
    dices = []
    for i, c in enumerate(classes):
        confMat = confMats[i]
        i = np.diag(confMat)
        u = confMat.sum(0)+confMat.sum(1)-i
        ious = i/u
        dice = 2*i/(confMat.sum(0)+confMat.sum(1))
        dices.append(dice[1])
    source[r_name]=np.asarray(dices)
    runs.append(r_name)

x = [ (classe, model) for classe in classes for model in runs ]
counts = sum(zip(*(source[_] for _ in runs)), ()) # like an hstack

source = ColumnDataSource(data=dict(x=x, counts=counts))

p = figure(x_range=FactorRange(*x), plot_height=500, plot_width=1200, title="Model's Dice by lesion",
           toolbar_location=None, tools="", y_range=(0.5, 0.75))

p.vbar(x='x', top='counts', width=0.9, source=source, line_color='white',
       fill_color=factor_cmap('x', palette=Spectral11, factors=runs, start=1, end=3)
      )

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None
output_file('per_lesion_dice.html')

show(p)
